# Project Setup
* This section describes how to create and configure a project

In [7]:
from labelbox import Client, Project, LabelingFrontend
from labelbox.schema.ontology import Tool, OntologyBuilder
client = Client()

### Identify project, dataset, and ontology

In [5]:
ontology_builder = OntologyBuilder(tools = [
    Tool(name = "cat", tool = Tool.Type.BBOX)
])
project = client.create_project(name = "my_new_project")
dataset = client.create_dataset(name = "my_new_dataset")
#Add data_rows (see basics/dataset to do this)

In [8]:
#Unless you are using a custom editor you should always use the following editor:
editor = next(client.get_labeling_frontends(where=LabelingFrontend.name == "Editor"))

In [11]:
#Note that you can use any dataset or ontology even if they already exist.
existing_project = client.get_project("ckm4xyfncfgja0760vpfdxoro")
ontology = existing_project.ontology()

### Setup and attach dataset

In [9]:
project.setup(editor, ontology_builder.asdict())
# Could also do if ontology is a normalized ontology
# project.setup(editor, ontology.normalized)

In [10]:
project.datasets.connect(dataset)

### Review

In [16]:
#Note setup_complete will be None if it fails.
print(project.setup_complete)
print(project.ontology)
print([ds.name for ds in project.datasets()])

2021-03-17 12:35:00+00:00
['my_new_dataset']
